### final

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'

# 기준월
base_ym = '202209'

### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 배후지 인구 1인당 상업용 부동산 공급면적
sql = f'''
with date as (
	select {base_ym}::text as base_ym
)
select (select base_ym from date) as base_ym
	 , A.pnu
	 , substring(A.pnu, 1, 8) as emd_cd
	 , case when sum(nvl(C.pnu_pop_cnt, 0) + nvl(D.pnu_pop_cnt, 0)) = 0 then 0 else round(1.0 * sum(nvl(E.tot_area, 0)) / sum(nvl(C.pnu_pop_cnt, 0) + nvl(D.pnu_pop_cnt, 0)), 4) end as commercial_per_population
from (
	select pnu, poly
	from m2.cremao_land
	where 1=1
		and jimok = '대'
) A -- 지목 '대' 토지만 추출 -> 모집단
left join (
	select pnu, poly
	from m2.cremao_land
) B -- 모집단 PNU 기준 반경 100m 내 PNU 모두 가져오기
	on ST_DistanceSphere(ST_Centroid(A.poly::geometry), ST_Centroid(B.poly::geometry)) < 100
left join m2.live_pop C -- 반경 100m 거주인구
	on B.pnu = C.pnu
left join m2.work_pop D -- 반경 100m 직장인구
	on B.pnu = D.pnu
left join (
	select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
		 , sum(tot_area) as tot_area
	from m1.building_permit_info
	where 1=1
		and arch_pms_day between (select dateadd(month, -11, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근1년 조건
		and (main_purps_cd_nm like '%업무%' or main_purps_cd_nm like '%근린%') -- 상업, 업무용 부동산 조건
		and arch_gb_cd_nm in ('신축','증축','재축') -- 신축, 중축, 재건축만 가져오기
	group by 1
) E -- 반경 100m 상업용부동산 공급 연면적SUM
	on B.pnu = E.pnu
group by 1, 2, 3
order by 1, 2, 3
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['base_ym','pnu','emd_cd','commercial_per_population']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y%m%d")
df['update_at'] = datetime.today().strftime("%Y%m%d")
df['work_user'] = 'du.Park'

In [ ]:
df

In [ ]:
df.drop(columns=['base_ym'])

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

###### ver1 (법정동 단위)

In [38]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 법정동별 상가 면적
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']
df_f_tot = df_f_tot[~df_f_tot['f_tot'].isnull()].reset_index(drop=True)

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 상가면적 / 거주인구
df['commercial_per_population'] = (df['total_count'] / df['f_tot'])
df = df[['emd_cd','commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_commercial_per_population;
    create table m2.datadam_commercial_per_population (    
    emd_cd varchar(10)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 18:08:47.688140  :  0
2022-10-30 18:08:50.007249  :  10000
17296


In [37]:
df.describe()

,commercial_per_population
count,17296.000000
mean,0.051612
std,0.292162
min,0.000000
25%,0.000259
50%,0.003113
75%,0.020263
max,17.100000


###### ver2

In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
  and base_ym = '202209'
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu_x']
df_bld_rgst['bun'] = df_bld_rgst['pnu_x'].str[:15] # 15: 본번까지 구분해서 병합
df_bld_rgst['emd_cd'] = df_bld_rgst['pnu_x'].str[:8] + '00'

# pnu별 유동인구 
sql = '''
select substring(bd_mgt_sn, 0, 20) as pnu, avg(f_tot)
  from m1.kt_bldg_xy_info
 where base_ym = '202209'
 group by substring(bd_mgt_sn, 0, 20)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu','f_tot']
df_f_tot['bun'] = df_f_tot['pnu'].str[:15]


# 데이터 프레임 병합
df = pd.merge(df_bld_rgst, df_f_tot, on='bun', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['pnu'].isnull()]

# 상업용 부동산 수 / 유동인구
df['commercial_per_population'] = (df['bld_count'] / df['f_tot'])

# inf 값 제거후 컬럼 정리
df = df[~(df['commercial_per_population']==np.inf)]
df = df[['pnu','emd_cd','commercial_per_population']]
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_commercial_per_population'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-05 12:00:48.330741  :  0
2022-11-05 12:00:54.061601  :  10000
2022-11-05 12:00:57.155262  :  20000
2022-11-05 12:01:03.401182  :  30000
2022-11-05 12:01:06.997267  :  40000
2022-11-05 12:01:09.446444  :  50000
2022-11-05 12:01:15.038604  :  60000
2022-11-05 12:01:17.109433  :  70000
2022-11-05 12:01:25.022565  :  80000
2022-11-05 12:01:28.034510  :  90000
2022-11-05 12:01:32.200959  :  100000
2022-11-05 12:01:35.561445  :  110000
2022-11-05 12:01:44.041954  :  120000
2022-11-05 12:01:46.818251  :  130000
2022-11-05 12:01:51.259685  :  140000
2022-11-05 12:01:53.284257  :  150000
2022-11-05 12:01:58.557820  :  160000
2022-11-05 12:02:05.226975  :  170000
2022-11-05 12:02:07.259563  :  180000
2022-11-05 12:02:10.095917  :  190000
2022-11-05 12:02:14.287604  :  200000
2022-11-05 12:02:16.280123  :  210000
2022-11-05 12:02:27.892285  :  220000
2022-11-05 12:02:29.854560  :  230000
2022-11-05 12:02:32.931336  :  240000
2022-11-05 12:02:41.524905  :  250000
2022-11-05 12:02:43.536573

In [33]:
df_bld_rgst

,bld_count,pnu_x,bun,emd_cd
0,1,1111010100100030100,111101010010003,1111010100
1,1,1111010100100060003,111101010010006,1111010100
2,1,1111010100100150005,111101010010015,1111010100
3,1,1111010100100150046,111101010010015,1111010100
4,1,1111010100100300007,111101010010030,1111010100
...,...,...,...,...
2047183,1,5013032026115240006,501303202611524,5013032000
2047184,1,5013032026115240011,501303202611524,5013032000
2047185,2,5013032026115410004,501303202611541,5013032000
2047186,1,5013032026115710001,501303202611571,5013032000


In [38]:
df_f_tot

,pnu,f_tot,bun
0,1111010100100010000,2198691.000000,111101010010001
1,1111010100100030065,3988060.000000,111101010010003
2,1111010100100060003,1444137.000000,111101010010006
3,1111010100100150005,2198691.000000,111101010010015
4,1111010100100150009,2198691.000000,111101010010015
...,...,...,...
1339480,5013032026108430003,97462.000000,501303202610843
1339481,5013032026113380001,37262.000000,501303202611338
1339482,5013032026114350009,28705.000000,501303202611435
1339483,5013032026114810001,37262.000000,501303202611481


In [2]:
df

,pnu,emd_cd,commercial_per_population,create_at,update_at,work_user
0,1111010100100030065,1111010100,0.000000,2022-11-05,2022-11-05,du.Park
2,1111010100100500031,1111010100,0.000000,2022-11-05,2022-11-05,du.Park
3,1111010100100500057,1111010100,0.000000,2022-11-05,2022-11-05,du.Park
4,1111010100100500030,1111010100,0.000000,2022-11-05,2022-11-05,du.Park
5,1111010100100530011,1111010100,0.000000,2022-11-05,2022-11-05,du.Park
...,...,...,...,...,...,...
9734808,5013032026115240008,5013032000,0.000010,2022-11-05,2022-11-05,du.Park
9734809,5013032026115240006,5013032000,0.000010,2022-11-05,2022-11-05,du.Park
9734810,5013032026115240001,5013032000,0.000010,2022-11-05,2022-11-05,du.Park
9734811,5013032026115240008,5013032000,0.000010,2022-11-05,2022-11-05,du.Park


In [3]:
df['commercial_per_population'].median()

2.8820295944090933e-07

In [83]:
df.describe()

,commercial_per_population
count,462282.000000
mean,0.000857
std,0.032387
min,0.000000
25%,0.000007
50%,0.000015
75%,0.000057
max,18.000000
